In [1]:
from pysparkling import Context

Word Count
---------

In [2]:
counts = Context().textFile(
    '../README.rst'
).map(
    lambda line: ''.join(ch if ch.isalnum() else ' ' for ch in line)
).flatMap(
    lambda line: line.split(' ')
).map(
    lambda word: (word, 1)
).reduceByKey(
    lambda a, b: a + b
)
print(counts.collect())

[(u'', 683), (u'1', 1), (u'100k', 1), (u'20Chat', 1), (u'3', 3), (u'4', 1), (u'A', 3), (u'ACCESS', 2), (u'API', 6), (u'AWS', 3), (u'All', 2), (u'Assume', 1), (u'Context', 11), (u'Count', 1), (u'Examples', 1), (u'Features', 1), (u'File', 4), (u'For', 1), (u'Given', 1), (u'HISTORY', 2), (u'Hadoop', 1), (u'Handles', 1), (u'ID', 1), (u'If', 1), (u'Install', 1), (u'Instantiating', 1), (u'It', 1), (u'JVM', 2), (u'Join', 2), (u'KEY', 2), (u'Now', 1), (u'Parallelization', 1), (u'Plain', 1), (u'Pool', 3), (u'PySpark', 1), (u'Python', 1), (u'RDD', 6), (u'README', 1), (u'Resolves', 1), (u'S3', 1), (u'SECRET', 1), (u'Some', 1), (u'Spark', 2), (u'SparkContext', 1), (u'Specify', 1), (u'Supports', 1), (u'The', 5), (u'These', 1), (u'They', 1), (u'This', 4), (u'ThreadPoolExecutor', 1), (u'URI', 1), (u'Use', 2), (u'Word', 1), (u'You', 2), (u'a', 17), (u'access', 1), (u'advanced', 1), (u'all', 1), (u'also', 1), (u'alt', 1), (u'an', 2), (u'and', 20), (u'any', 2), (u'api', 3), (u'are', 2), (u'arguments', 1

## Distributed Confusion Matrix

In [11]:
import numpy as np

predicted_true = Context().parallelize([
    (0, 0), (1, 0), (2, 2), (1, 1), (1, 2), (2, 2),
    (0, 2), (1, 0), (2, 1), (1, 1), (0, 0), (0, 0),
])

def cm_per_pair(p_t):
    cm = np.zeros((3, 3))
    cm[p_t[0], p_t[1]] = 1
    return cm

predicted_true.aggregate(
    zeroValue=np.zeros((3, 3)),
    seqOp=lambda prev, p_t: prev + cm_per_pair(p_t),
    combOp=np.add,
)

array([[ 3.,  0.,  1.],
       [ 2.,  2.,  1.],
       [ 0.,  1.,  2.]])

Accessing Public Datasets on AWS S3
===================

You need to have valid AWS account credentials in your environment under ``AWS_ACCESS_KEY_ID`` and ``AWS_SECRET_ACCESS_KEY`` for the next demos.


Common Crawl
-----------

More info on the dataset is in this [blog post](http://blog.commoncrawl.org/2015/05/march-2015-crawl-archive-available/).

In [3]:
# read all the paths of warc and wat files of the latest Common Crawl
paths_rdd = Context().textFile(
    's3n://aws-publicdatasets/common-crawl/crawl-data/CC-MAIN-2015-11/warc.paths.*,'
    's3n://aws-publicdatasets/common-crawl/crawl-data/CC-MAIN-2015-11/wat.paths.gz'
)

print(paths_rdd.takeSample(5))

[u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936462555.21/warc/CC-MAIN-20150226074102-00158-ip-10-28-5-156.ec2.internal.warc.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936462982.10/warc/CC-MAIN-20150226074102-00095-ip-10-28-5-156.ec2.internal.warc.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936463181.39/warc/CC-MAIN-20150226074103-00261-ip-10-28-5-156.ec2.internal.warc.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936463956.95/wat/CC-MAIN-20150226074103-00277-ip-10-28-5-156.ec2.internal.warc.wat.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936465069.3/wat/CC-MAIN-20150226074105-00332-ip-10-28-5-156.ec2.internal.warc.wat.gz']


Human Biome Project
------------------

In [4]:
# access data from the Human Microbiome Project
by_subject_rdd = Context().textFile(
    's3n://human-microbiome-project/DEMO/HM16STR/46333/by_subject/*'
)
print(by_subject_rdd.takeSample(1))

[u'ACCAAGGCTTTGACGGGTAGCCGGCCTGAGAGGGTGACCGGCCACATTGGGACTGAGATA']
